# 生成不同晶格常数的FCC/HCP，bulk 和 12-layer film 结构，用于 DFT 计算

In [1]:
from ase.visualize import view
from ase.build import bulk
from mymetal.build.film.stretch import generate_film, stretch_list_along_direction_to_cell
from mymetal.build.film.extrfilm import my_extr_thick
from mymetal.io.vasp import my_write_vasp
from os import makedirs, path
from numpy import sqrt

# 公共晶格常数

In [2]:
general_lattice_constant = [2.8485, 2.839, 2.66866, 2.72544]
eq_a0_hcp = 2.8390
eq_a0_fcc = 2.8485
general_stretch_list_fcc = [x / eq_a0_fcc for x in general_lattice_constant]
general_stretch_list_hcp = [x / eq_a0_hcp for x in general_lattice_constant]
print(general_lattice_constant)
print(general_stretch_list_fcc)
print(general_stretch_list_hcp)

[2.8485, 2.839, 2.66866, 2.72544]
[1.0, 0.9966649113568544, 0.9368650166754432, 0.9567983149025803]
[1.0033462486791125, 1.0, 0.9400000000000001, 0.96]


# FCC

In [23]:
def biaxial_stretch():
    a0 = eq_a0_fcc
    a0_fcc = a0 * sqrt(2)
    film = generate_film(symbols = 'Au', structure = 'fcc', num_layers = 12, my_vacuum = 10, slice_plane = (1,1,1), a_fcc = a0_fcc)
    interlayer_dixtance = my_extr_thick(film)/22
    # 构建（111）基面的 FCC 块体
    bulk = generate_film(symbols = 'Au', structure = 'fcc', num_layers = 12, my_vacuum = 10, slice_plane = (1,1,1), a_fcc = a0_fcc)
    bulk.center(vacuum=interlayer_dixtance, axis=2)
    stretch_3_direction_list = []
    for i in general_stretch_list_fcc:
        stretch_3_direction_list.append([i, i, 1.0/i**2])
    for stretch_list in stretch_3_direction_list:
        print(stretch_list)

    films_stretch = stretch_list_along_direction_to_cell(film , stretch_3_direction_lists=stretch_3_direction_list)
    bulks_stretch  = stretch_list_along_direction_to_cell(bulk , stretch_3_direction_lists=stretch_3_direction_list)

    #print(np.array(film.get_cell()))
    format_type = '%.3f'
    for i, film_stretch in enumerate(films_stretch):
        formatted_i = format_type % stretch_3_direction_list[i][0]
        a = float('%.14f' % (a0 * stretch_3_direction_list[i][0]))
        film_stretch.center(vacuum=10, axis=2)
        filename1 = f'./fcc/{formatted_i}-{a}/full_relaxed_surface_111/POSCAR' 
        filename2 = f'./fcc/{formatted_i}-{a}/bulk/POSCAR' 
        makedirs(path.dirname(filename1), exist_ok=True)   
        makedirs(path.dirname(filename2), exist_ok=True)   
        my_write_vasp(filename1, film_stretch,     label = f'Au FCC thin film {formatted_i}', lattice_scale_factor=a)
        my_write_vasp(filename2, bulks_stretch[i], label = f'Au bulk (111)-base {formatted_i}', lattice_scale_factor=a)
        
biaxial_stretch()

[1.0, 1.0, 1.0]
[0.9966649113568544, 0.9966649113568544, 1.0067036947384473]
[0.9368650166754432, 0.9368650166754432, 1.1393206142354542]
[0.9567983149025803, 0.9567983149025803, 1.0923434187700165]


# HCP

In [24]:


def biaxial_stretch():
    a0 = eq_a0_hcp
    a0_hcp = a0 
    film = generate_film(symbols = 'Au', structure = 'hcp', num_layers = 12, my_vacuum = 10, slice_plane = (0,0,1), a_hcp = a0_hcp)
    interlayer_dixtance = my_extr_thick(film)/22
    # 构建（0001）基面的 HCP 块体
    bulk = generate_film(symbols = 'Au', structure = 'hcp', num_layers = 12, my_vacuum = 10, slice_plane = (0,0,1), a_hcp = a0_hcp)
    bulk.center(vacuum=interlayer_dixtance, axis=2)
    stretch_3_direction_list = []
    for i in general_stretch_list_hcp:
        stretch_3_direction_list.append([i, i, 1.0/i**2])
    for stretch_list in stretch_3_direction_list:
        print(stretch_list)

    films_stretch = stretch_list_along_direction_to_cell(film , stretch_3_direction_lists=stretch_3_direction_list)
    bulks_stretch  = stretch_list_along_direction_to_cell(bulk , stretch_3_direction_lists=stretch_3_direction_list)

    #print(np.array(film.get_cell()))
    format_type = '%.3f'
    for i, film_stretch in enumerate(films_stretch):
        formatted_i = format_type % stretch_3_direction_list[i][0]
        a = float('%.14f' % (a0 * stretch_3_direction_list[i][0]))
        film_stretch.center(vacuum=10, axis=2)
        filename1 = f'./hcp/{formatted_i}-{a}/full_relaxed_surface_0001/POSCAR' 
        filename2 = f'./hcp/{formatted_i}-{a}/bulk/POSCAR' 
        makedirs(path.dirname(filename1), exist_ok=True)   
        makedirs(path.dirname(filename2), exist_ok=True)   
        my_write_vasp(filename1, film_stretch,     label = f'Au HCP thin film {formatted_i}', lattice_scale_factor=a)
        my_write_vasp(filename2, bulks_stretch[i], label = f'Au bulk (0001)-base {formatted_i}', lattice_scale_factor=a)
        
        
biaxial_stretch()

[1.0033462486791125, 1.0033462486791125, 0.9933409455299664]
[1.0, 1.0, 1.0]
[0.9400000000000001, 0.9400000000000001, 1.131733816206428]
[0.96, 0.96, 1.0850694444444444]
